In [1]:
import riva.client

In [2]:
import soundfile as sf

In [10]:
import grpc
import io
import IPython.display as ipd
import wave

In [11]:
auth = riva.client.Auth(uri='localhost:50051')
riva_asr = riva.client.ASRService(auth)

In [13]:
AUDIO_FILE = 'harvard.wav'
with wave.open(AUDIO_FILE, "rb") as wav:
    audio = wav.readframes(wav.getnframes())
# ipd.Audio(audio, rate=16000)

In [17]:
config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",  
    max_alternatives=1,
    enable_automatic_punctuation=True,
    audio_channel_count = 1   
)

In [18]:
response = riva_asr.offline_recognize(audio, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript:", asr_best_transcript)

print("\n\nFull Response Message:")
print(response)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Error: Unavailable model requested given these parameters: language_code=vi; sample_rate=16000; type=offline; "
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:50051 {created_time:"2025-11-12T18:27:47.3852327+00:00", grpc_status:3, grpc_message:"Error: Unavailable model requested given these parameters: language_code=vi; sample_rate=16000; type=offline; "}"
>

In [5]:
import riva.client
from riva.client import ASRService
import wave

# Địa chỉ server
RIVA_SERVER = "localhost:50051"

# Kết nối Riva
auth = riva.client.Auth(uri=RIVA_SERVER)
riva_asr = ASRService(auth)

# File âm thanh test (16kHz, mono, wav)
AUDIO_FILE = "harvard.wav"

with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# Cấu hình ASR
config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)
print("🗣️  Kết quả nhận dạng:")
def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]

# Gọi inference
responses = riva_asr.streaming_response_generator(
    audio_chunks(audio_bytes),
    config,
    end_of_stream=True  # ⚠️ báo audio đã hết
)
print("🗣️  Kết quả nhận dạng:")
for response in responses:
    for result in response.results:
        if len(result.alternatives) > 0:
            print("Transcript:", result.alternatives[0].transcript)

🗣️  Kết quả nhận dạng:


TypeError: ASRService.streaming_response_generator() got an unexpected keyword argument 'end_of_stream'

In [8]:
import wave
import riva.client
from riva.client import ASRService

# Kết nối server
RIVA_SERVER = "localhost:50051"
auth = riva.client.Auth(uri=RIVA_SERVER)
riva_asr = ASRService(auth)

# Đọc file WAV 16kHz mono
AUDIO_FILE = "harvard.wav"
with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# Generator chia chunk và báo end-of-stream
def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]
    yield b""  # ⚠️ kết thúc streaming

# Cấu hình ASR
config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)

# Gọi streaming inference
responses = riva_asr.streaming_response_generator(audio_chunks(audio_bytes), config)

print("🗣️  Kết quả nhận dạng:")



🗣️  Kết quả nhận dạng:


In [9]:
responses

<generator object ASRService.streaming_response_generator at 0x0000022A94CD2110>

In [1]:
import wave
from riva.client import ASRService, RecognitionConfig, AudioEncoding, Auth

# Kết nối server
auth = Auth(uri="localhost:50051")
riva_asr = ASRService(auth)

# Đọc audio
AUDIO_FILE = "harvard.wav"
with wave.open(AUDIO_FILE, "rb") as wav:
    audio_bytes = wav.readframes(wav.getnframes())

# Generator cho streaming
def audio_chunks(audio_bytes, chunk_size=4000):
    for i in range(0, len(audio_bytes), chunk_size):
        yield audio_bytes[i:i+chunk_size]

# Cấu hình ASR
config = RecognitionConfig(
    encoding=AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=16000,
    language_code="vi-VN",
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False
)

# Streaming inference
responses = riva_asr.streaming_response_generator(audio_chunks(audio_bytes), config)

# Kiểm tra output
for response in responses:
    for result in response.results:
        for alt in result.alternatives:
            transcript = alt.transcript
            speaker = getattr(alt, "speaker_tag", None)
            if speaker:
                print(f"[Speaker {speaker}]: {transcript}")
            else:
                print(f"[Transcript]: {transcript}")


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception iterating requests!"
	debug_error_string = "None"
>